# Trabalho - Etapa Final

### GitHub: https://github.com/maviless/mc536-projeto

## Duplas MAP e RAD
## Membros:
* Marcelo Martins Vilela Filho  - RA: 202619
* Pedro Rodrigo Ramos Morelli   - RA: 204737
* Renan Clarindo Amorim         - RA: 186454

## Primeira Parte: Esquema relacional com SQL

#### Fontes dos datasets utilizados:
* Hospital General Information (hospital.csv), acesso em: https://data.medicare.gov/Hospital-Compare/Hospital-General-Information/xubh-q36u
* Physician Compare National Downloadable File (doctor_mini.csv, reduzido do tamanho original de 700mb), acesso em: https://data.medicare.gov/Physician-Compare/Physician-Compare-National-Downloadable-File/mj5m-pzi6

### Modelo Conceitual:
![SQL](esquema_conceitual_sql_trabfinal.jpeg)
### Modelo Lógico:
![SQL2](esquema_relacional_sql_trabfinal.jpeg)

In [ ]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Medicos;
DROP TABLE IF EXISTS Hospitais;


CREATE TABLE Hospitais (
    id VARCHAR(7) NOT NULL,
    facility_name VARCHAR(200),
    city VARCHAR(30),
    county VARCHAR(30),
    emergency_services VARCHAR(5),
    patient_experience VARCHAR(100),
    mortality VARCHAR(100),
    safety_of_care VARCHAR(100),
    readmission VARCHAR(100),
    effectiveness_of_care VARCHAR(100),
    overall_rating VARCHAR(20),
    PRIMARY KEY (id)
) AS SELECT 
    "Facility ID",
    "Facility Name",
    City,
    "County Name",
    "Emergency Services",
    "Patient experience national comparison",
    "Mortality national comparison",
    "Safety of care national comparison",
    "Readmission national comparison",
    "Effectiveness of care national comparison",
    "Hospital overall rating"
FROM CSVREAD('hospital.csv');

CREATE TABLE Medicos (
    npi VARCHAR(15) NOT NULL,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(2),
    specialty VARCHAR(50),
    city VARCHAR(30),
    hospital VARCHAR(200)
) AS SELECT DISTINCT
    NPI,
    "First Name",
    "Last Name",
    Gender,
    "Primary specialty",
    City,
    "Hospital affiliation LBN 1"
FROM CSVREAD('doctor_mini.csv');

* Query 1: Selecionada uma especialidade, listar os hospitais com mais médicos daquela especialidade

(Expansão da Query 1 necessita de lógica relacional diferente, hierárquica.

Ex: Para cada especialidade, listar o hospital com maior número de médicos que atendem aquela especialidade.
)

In [3]:
SELECT h.facility_name, m.specialty, COUNT (*) aux
FROM Medicos m, Hospitais h
WHERE m.specialty = 'NEUROLOGY' AND h.facility_name = m.hospital
GROUP BY h.facility_name
ORDER BY aux DESC

* Query 2: Listar os hospitais com mais médicos

In [4]:
SELECT Medicos.hospital, COUNT(*) aux
    FROM Medicos, hospitais
    WHERE Medicos.hospital = hospitais.facility_name
    GROUP BY Medicos.hospital
    ORDER BY aux DESC

* Query 3: Listar os 10 melhores hospitais com alguma especialidade

In [5]:
SELECT h.facility_name, h.city
    FROM Hospitais h, Medicos m
    WHERE m.specialty = 'PATHOLOGY' 
    and m.hospital = h.facility_name 
    and h.overall_rating != 'Not Available'
    ORDER BY h.overall_rating DESC 
    LIMIT 10

* Query 4: Listar as cidades com mais hospitais

In [6]:
SELECT city, COUNT(*) qtd
    FROM Hospitais
    GROUP BY city
    ORDER BY qtd DESC

* Query 5: Listar os médicos que trabalham nos hospitais com melhor experiência para o paciente

In [7]:
SELECT DISTINCT m.first_name, m.last_name, m.specialty
    FROM Medicos m, Hospitais h
    WHERE m.hospital = h.facility_name 
    and h.patient_experience = 'Above the national average'

## Segunda Parte: Esquema hierárquico com XQuery

#### Fontes dos datasets utilizados:
* Hospital General Information (hospital2.xml), acesso em: http://www.students.ic.unicamp.br/~ra186454/hospital2.xml
* Physician Compare National Downloadable File (doctor2_mini.xml, reduzido do tamanho original de 700mb), acesso em: http://www.students.ic.unicamp.br/~ra186454/doctor2_mini.xml

### Modelo Conceitual:
![XQuery1](esquema_conceitual_xquery_trabfinal.jpeg)
### Modelo Lógico:
![XQuery2](esquema_relacional_sql_trabfinal.jpeg)

* Query 1: Contagem do número de médicos por especialidade

In [ ]:
let $doctors := doc('http://www.students.ic.unicamp.br/~ra186454/doctor2_mini.xml')
for $i in ($doctors//especialidade)
for $j in $i//count(medico)
return if ($j != 0) then ($i/@nome/string(), $j) else ()

* Query 2: Seleciona os hospitais de excelência (melhores parâmetros de avaliação) e os lista por ordem descendente da melhor classificação

In [ ]:
let $hospital := doc('http://www.students.ic.unicamp.br/~ra186454/hospital2.xml')
for $x in $hospital/estados/estado/cidades/cidade/hospitais/hospital
where $x/mortality="&#34;Below the national average&#34;" and $x/safety="&#34;Above the national average&#34;" 
and $x/effectiveness="&#34;Above the national average&#34;"
order by $x/hospital_rating descending 
return ( "  - Nome do Hospital" ,$x/facility_name/string(), " Rating:" ,$x/hospital_rating/string())

* Query 3: Retorna o nome dos médicos que operam em hospitais cuja efetividade é acima da média nacional

In [ ]:
let $hospital := doc('http://www.students.ic.unicamp.br/~ra186454/hospital2.xml')
let $doctors := doc('http://www.students.ic.unicamp.br/~ra186454/doctor2_mini.xml')

for $x in $hospital/estados/estado/cidades/cidade/hospitais/hospital
for $y in $doctors/especialidades/especialidade/medicos/medico
where $x/effectiveness="&#34;Above the national average&#34;" and $x/facility_name=$y/hospital1
return ("Dr.", $y/terceiro_nome/string(), ", ", $y/primeiro_nome/string() , ".  ")

* Query 4: Para cada especialidade, listar o hospital com maior número de médicos que atendem aquela especialidade

In [ ]:
let $hospital := doc('http://www.students.ic.unicamp.br/~ra186454/hospital2.xml')
let $doctors := doc('http://www.students.ic.unicamp.br/~ra186454/doctor2_mini.xml')

for $x in $doctors//especialidades
for $y in distinct-values($x//hospital1)

let $count := count($x//medico[@hospital1 eq $y])
order by $count descending

return concat("("," ", $y," ",$count, ")")

* Query 5: Para cada cidade, listar qual a especialidade com mais médicos

In [ ]:
let $hospital := doc('http://www.students.ic.unicamp.br/~ra186454/hospital2.xml')
let $doctors := doc('http://www.students.ic.unicamp.br/~ra186454/doctor2_mini.xml')


for $y in distinct-values($doctors//especialidade)
for $x in $doctors//cidade

let $count := count($x//medico[@cidade eq $y/@nome])
order by $count descending

return concat("("," ", $y/@nome," ",$count, ")")

## Terceira Parte: Esquema de redes

#### Fontes dos datasets utilizados:
* Hospital General Information (hospital.csv), acesso em: https://data.medicare.gov/Hospital-Compare/Hospital-General-Information/xubh-q36u
* Physician Compare National Downloadable File (doctor_mini.csv, reduzido do tamanho original de 700mb), acesso em: https://data.medicare.gov/Physician-Compare/Physician-Compare-National-Downloadable-File/mj5m-pzi6
* World Cities Database ()

### Modelo Conceitual:
![Redes](esquema_conceitual_redes_trabfinal.jpeg)

### Esquema dos gráficos:
![Redes2](grafo_hospitais.jpg)

## Criação de índices

In [ ]:
CALL apoc.schema.assert(
 {County:['name'],City:['name'],ZipCode:['name'],Address:['name']},
 {Hospital:['id'],State:['name']});

In [ ]:
LOAD CSV WITH HEADERS FROM "http://www.students.ic.unicamp.br/~ra186454/hospital.csv" as row
MERGE (state:State{name:row.State})
MERGE (state)<-[:IS_IN]-(county:County{name:row.`County Name`})
MERGE (county)<-[:IS_IN]-(city:City{name:row.City})
MERGE (city)<-[:IS_IN]-(zip:ZipCode{name:row.`ZIP Code`})
MERGE (zip)<-[:IS_IN]-(address:Address{name:row.Address})
MERGE (h:Hospital{id:row.`Provider ID`})
ON CREATE SET h.phone=row.`Phone Number`,
              h.emergency_services = row.`Emergency Services`,
              h.name= row.`Hospital Name`,
              h.mortality = row.`Mortality national comparison`,
              h.safety = row.`Safety of care national comparison`,
              h.timeliness = row.`Timeliness of care national comparison`,
              h.experience = row.`Patient experience national comparison`,
              h.effectiveness = row.`Effectiveness of care national comparison`
MERGE (h)-[:IS_IN]->(address)
MERGE (type:HospitalType{name:row.`Hospital Type`})
MERGE (h)-[:HAS_TYPE]->(type)
MERGE (ownership:Ownership{name: row.`Hospital Ownership`})
MERGE (h)-[:HAS_OWNERSHIP]->(ownership)
MERGE (rating:Rating{name:row.`Hospital overall rating`})
MERGE (h)-[:HAS_RATING]->(rating)

In [ ]:
LOAD CSV WITH HEADERS FROM "http://www.students.ic.unicamp.br/~ra186454/worldcities.csv" as row
MATCH (hospital:Hospital {id:row.id})
SET hospital.latitude = row.latitude, hospital.longitude = row.longitude

* Query 1: Simulação dos 10 hospitais mais próximos de Liberty Island

In [ ]:
with "1 Liberty Island, New York" as myLocation
call apoc.spatial.geocodeOnce(myLocation) yield location
WITH point({longitude: location.longitude, latitude: location.latitude}) as myPosition,100 as distanceInKm
MATCH (h:Hospital)-->(rating:Rating)
WHERE exists(h.latitude) and
distance(myPosition, point({longitude:h.longitude,latitude:h.latitude})) < (distanceInKm * 100)
RETURN h,rating.name as rating,distance(myPosition, point({longitude:h.longitude,latitude:h.latitude})) as distance order by distance limit 10

* Query 2: Retornar os melhores 10 estados por avaliação

In [ ]:
MATCH (r)<-[:HAS_RATING]-(h:Hospital)-[:IS_IN*5..5]->(state)
where not r.name="Not Available"
return state.name as state,avg(toINT(r.name)) as averageRating,count(h) as numberOfHospitals order by averageRating desc limit 15

* Query 3: Número de hospitais por cidade

In [ ]:
MATCH (h:Hospital)-[:IS_IN*3..3]->(city)
return city.name as city,count(h) as NumberOfHospitals order by NumberOfHospitals desc limit 15

* Query 4: Estados com hospitais acima da média em efetividade de tratamento

In [ ]:
MATCH (h:Hospital)-[:IS_IN*5..5]->(state) where h.effectiveness = "Above the National average"
return state.name as state,h.effectiveness,count(h) as numberOfHospitals order by numberOfHospitals desc limit 15

* Query 5: Estados com hospitais abaixo da média em mortalidade

In [ ]:
MATCH (h:Hospital)-[:IS_IN*5..5]->(state) where h.mortality = "Below the National average"
return state.name as state,h.mortality,count(h) as numberOfHospitals order by numberOfHospitals desc limit 15